In [1]:
# Program: flappy_bird.ipynb
# Purpose: Recreate Flappy Bird to practice 3D Modeling and general VPython
# Author:  Tobias Safie - tks57@drexel.edu
# Date:    November 6, 2024
from vpython import *
import random

g = -9.8  # gravity (m/s^2)
flap_force = 5  # flap velocity (m/s)
t = 0  # time
dt = 0.01  # time step
pipe_gap = 1.75  # standard gap
pipe_speed = 3  # standard speed
scale_factor = 0.1
running = True
waiting = True

# Game window setup
scene.width = 350
scene.height = 350
scene.center = vec(1.5, 0, 0)  # Center view to keep bird in center
scene.range = 5
scene.background = vec(0.1, 0.1, 0.4)

# Lock the window
scene.userzoom = False
scene.userspin = False
scene.autoscale = False

# Bird model
eye = sphere(pos=vec(-0.1, 1, 0.75), radius=1.2, color=color.white, opacity=1, shininess=1, emissive=True)
eyeball = ellipsoid(pos=vec(1.4, 1.1, 2), length=0.3, height=0.7, width=0.15, color=color.black, emissive=True)
pupil = sphere(pos=vec(3, 0.73, 2), radius=0.1, color=color.white, opacity=1, shininess=1, emissive=True)
mouthtop = ellipsoid(pos=vec(1.5, 0, 0), length=4.5, height=1.5, width=3, color=color.red, axis=vec(1, 0.2, 0), shininess=1)
mouthbottom = ellipsoid(pos=vec(1.5, -1, 0), length=4.5, height=1.5, width=3, color=color.red, axis=vec(1, -0.2, 0), shininess=1)
body = ellipsoid(pos=vec(-4, -1, -1.5), length=8, height=5, width=5, color=color.yellow, shininess=0.2, emissive=True)
bodytop = sphere(pos=vec(-3.7, -0.1, -2.5), radius=3, color=color.yellow, shininess=0.2, emissive=True)
wing1 = ellipsoid(pos=vec(-3.8, 2.95, 2.95), length=4.2, height=0.95, width=1, color=color.orange, shininess=1, axis=vec(-0.35, 1, 0))
wing2 = ellipsoid(pos=vec(-4.7, 1.95, 3), length=3.95, height=1, width=1, color=color.orange, shininess=1, axis=vec(1, -0.6, 0))
wing3 = ellipsoid(pos=vec(-4.5, 1.05, 3), length=3.15, height=0.9, width=1, color=color.orange, shininess=1, axis=vec(1, 0, 0))

bird = compound([eye, eyeball, pupil, mouthtop, mouthbottom, body, bodytop, wing1, wing2, wing3])
bird.pos = vec(-2, 0, 0)  # Initial position
bird.v = vec(0, 0, 0)  # Initial velocity

# Scale the bird down
bird.radius = 5 * scale_factor  # Approximate radius for collision detection
bird.size = bird.size * scale_factor

# Pipe creation
def make_pipes(x_pos):
    a = 100  #define 'a' for color calculations
    gap_y = random.uniform(-2, 2)  #random position for gap

    # Caps and poles for top pipe
    top_pipe_cap = cylinder(pos=vec(x_pos, gap_y + pipe_gap / 2 + 0.5, 0), radius=1, size=vec(1, 2, 1), color=vec(44/a, 176/a, 26/a), axis=vec(0, 1, 0), shininess=1)
    top_pipe_pole = cylinder(pos=vec(x_pos, gap_y + pipe_gap / 2 + 1.5, 0), radius=0.8, size=vec(4, 1, 1), color=vec(44/a, 176/a, 26/a), axis=vec(0, 1, 0), shininess=1)

    # Caps and poles for bottom pipe
    bottom_pipe_cap = cylinder(pos=vec(x_pos, gap_y - pipe_gap / 2 - 0.5, 0), radius=1, size=vec(1, 2, 1), color=vec(44/a, 176/a, 26/a), axis=vec(0, -1, 0), shininess=1)
    bottom_pipe_pole = cylinder(pos=vec(x_pos, gap_y - pipe_gap / 2 - 1.5, 0), radius=0.8, size=vec(4, 1, 1), color=vec(44/a, 176/a, 26/a), axis=vec(0, -1, 0), shininess=1)

    # Return the pipe components
    return (top_pipe_cap, top_pipe_pole), (bottom_pipe_cap, bottom_pipe_pole)

# Initialize pipes
pipes = [make_pipes(10), make_pipes(15)]

# Flap function
def flap():
    global waiting
    if waiting:
        waiting = False
        bird.v.y = flap_force
    else:
        bird.v.y = flap_force

# Bind flap to w
scene.bind('keydown', lambda evt: flap() if evt.key == 'w' else None)

# Game variables
score = 0
running = True

# Wait until w is pressed
while waiting:
    rate(50)

# Title
scene.title = "Press w to start"

# Game loop
while running:
    rate(1/dt)  # Frame rate based on dt

    # Implement gravity
    bird.v.y += g * dt
    bird.pos += bird.v * dt

    for top_pipe, bottom_pipe in pipes:
        # Move the pipes
        top_pipe[0].pos.x -= pipe_speed * dt  # Top pipe cap
        top_pipe[1].pos.x -= pipe_speed * dt  # Top pipe pole
        bottom_pipe[0].pos.x -= pipe_speed * dt  # Bottom pipe cap
        bottom_pipe[1].pos.x -= pipe_speed * dt  # Bottom pipe pole

        # Check collisions
        if (abs(bird.pos.x - top_pipe[0].pos.x) < 0.4 + bird.radius) and (bird.pos.y > top_pipe[0].pos.y or bird.pos.y < bottom_pipe[0].pos.y):
            running = False  # End game

        # Reset pipes when they go offscreen
        if top_pipe[0].pos.x < -4.5:
            top_pipe[0].pos.x += 15
            top_pipe[1].pos.x += 15
            bottom_pipe[0].pos.x += 15
            bottom_pipe[1].pos.x += 15
            score += 1

    # Check collision with ground
    if bird.pos.y < -5 + bird.radius:
        running = False

    # Display score
    scene.caption = f"Score: {score}"

<IPython.core.display.Javascript object>